In [1]:
import base64
import numpy as np
import cv2
import imageio

In [3]:
def decode_image(base64_data):
    base64_data = base64_data.split(",")[1]
    image_data = base64.b64decode(base64_data)
    np_data = np.frombuffer(image_data, np.uint8)
    image = cv2.imdecode(np_data, cv2.IMREAD_COLOR)
    return image

In [4]:
def cv_close():
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [5]:
def resize(base64_data, dimensions):
    image = decode_image(base64_data)
    # cv2.imshow("Original Image", image)
    resized_image = cv2.resize(image, dimensions, interpolation=cv2.INTER_LINEAR)
    # cv2.imshow("Resized Image", resized_image)
    # cv_close()
    return resized_image

In [19]:
def rotate(base64_data, angle):
    image = decode_image(base64_data)
    h, w = image.shape[:2]
    center = (w // 2, h // 2)
    mat = cv2.getRotationMatrix2D(center, angle, 1)
    cos = np.abs(mat[0, 0])
    sin = np.abs(mat[0, 1])
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))
    mat[0, 2] += (new_w / 2) - center[0]
    mat[1, 2] += (new_h / 2) - center[1]
    rotated_image = cv2.warpAffine(image, mat, (new_w, new_h), borderMode=cv2.BORDER_CONSTANT, borderValue=(255,255,255))
    cv2.imshow("Original Image", image)
    cv2.imshow("Rotated Image", rotated_image)
    cv_close()

In [24]:
def grayscale(base64_data):
    image = decode_image(base64_data)
    cv2.imshow("Original Image", image)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cv2.imshow("Grayscale Image", gray_image)
    cv_close()

In [11]:
def to_rgb(base64_data):
    image = decode_image(base64_data)
    cv2.imshow("Original Image", image)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    cv2.imshow("RGB Color", rgb_image)
    cv_close()

In [12]:
def to_bgr(base64_data):
    image = decode_image(base64_data)
    cv2.imshow("Original Image", image)
    bgr_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    cv2.imshow("BGR Color", bgr_image)
    cv_close()

In [8]:
def perspective_transform(base64_data, coordinates):
    image = decode_image(base64_data)
    h, w = image.shape[:2]
    tl = np.array(coordinates[0])
    bl = np.array(coordinates[1])
    tr = np.array(coordinates[2])
    br = np.array(coordinates[3])       
    print(h, w)
    pts1 = np.array([tl, bl, tr, br], np.float32)
    pts2 = np.array([[0, 0], [0, h], [w, 0], [w, h]], dtype=np.float32)
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    
    cv2.circle(image, tl, 5, (0,0,255), -1)
    cv2.circle(image, bl, 5, (0,0,255), -1)
    cv2.circle(image, tr, 5, (0,0,255), -1)
    cv2.circle(image, br, 5, (0,0,255), -1)
    
    transformed_image = cv2.warpPerspective(image, matrix, (w, h))
    cv2.imshow("Original Image", image)
    # if transformed_image:
    cv2.imshow("Transformed Image", transformed_image)
    
    cv_close()

In [ ]:
def compress_image(base64_data, quality=15):
    image = decode_image(base64_data)
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality]
    _, encoded_img = cv2.imencode('.jpg', image, encode_param)
    
    decoded_img = cv2.imdecode(encoded_img, cv2.IMREAD_COLOR)
    cv2.imshow("Original Image", image)
    cv2.imshow("Compressed Image", decoded_img)
    
    cv_close()

In [ ]:
def generate_gif(images, frame_interval):
    pass
cap = cv2.VideoCapture("test.mp4")
frames = []
count = 0
while True:    
    ret, frame = cap.read()
    cv2.imshow("Captured Frame", frame)
    key = cv2.waitKey(0)
    if key == ord('a'):
        count += 1
        print(count, ": Adding to the frames")
        frames.append(frame)
    elif key == ord("q"):
        cv2.destroyAllWindows()
        break

print("SAVING")
with imageio.get_writer("test.gif", mode="I") as writer:
    for frame in frames:
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        writer.append_data(rgb_frame)
        

Save the gif


In [ ]:
rotate(
    """
    ...
    """, 45)

In [ ]:
# to_rgb(bgr)